In [15]:
import pandas as pd

# 创建DataFrame
df = pd.read_csv('result.csv')

# 转换日期列为 datetime 格式，并提取年月日
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

# 拆分 'location' 列为多个行政区级别
split_columns = df['location'].str.split(' ', expand=True)

# 定义最大列数（5个层次：省、市、区县、乡镇、村或其他）
max_columns = 5
split_columns = split_columns.reindex(columns=range(max_columns))

# 处理重庆市开头的情况
# 如果第一列是 '重庆市'，则设置 city 列为 None，后面的列依次后移
split_columns.loc[split_columns[0] == '重庆市', 1:] = split_columns.loc[
    split_columns[0] == '重庆市', 0:max_columns - 1
].shift(1, axis=1)
split_columns.loc[split_columns[0] == '重庆市', 1] = None  # 设置 city 列为 None

# 设置列名
location_columns = ['province', 'city', 'district', 'town', 'village']
split_columns.columns = location_columns

# 将拆分结果与原DataFrame合并，并删除原始 'location' 列
df = pd.concat([df, split_columns], axis=1)
df.drop(columns='location', inplace=True)

# 显示处理后的 DataFrame
df.to_csv('final_result.csv')